# Update Scheduled Scans

This notebook finds discovery runs whose schedule is "Every day, starting at HH:00 every 8 hours until complete" and updates them to every 12 hours, preserving the original start hour (e.g., 01:00 → [01:00, 13:00]).

- Preview the proposed changes first.
- Toggle `APPLY_CHANGES = True` to patch schedules on the appliance.
- Schedules not matching the exact 8-hour cadence are left unchanged.

## Requirements
Uses `requests`, `pandas`, and `PyYAML`. If needed, uncomment below to install.

In [ ]:
# %pip install -q requests pandas pyyaml
import pandas as pd
import requests
import yaml
from pathlib import Path
from urllib.parse import urljoin
import json, os

## Select Appliance and Apply Toggle
Set one of `APPLIANCE_NAME` or `APPLIANCE_INDEX` if you have multiple entries in `config.yaml`. Leave as-is to use the first appliance.

Set `APPLY_CHANGES = True` to perform the updates. When `False`, the notebook previews the changes only.

In [ ]:
APPLIANCE_NAME = None   # e.g., 'prod' or 'dev'
APPLIANCE_INDEX = 0     # integer index if not using name selection
#APPLIANCE_INDEX = 1
APPLY_CHANGES  = True  # toggle to True to PATCH schedules

## Load configuration (from ../config.yaml)
This mirrors the other DisMAL notebooks: it resolves `target`, `token` or `token_file`, optional `api_version` and `verify_ssl`, and prepares an output folder (`output_<target>`).

In [ ]:
def _find_repo_root(start: Path) -> Path:
    for p in [start] + list(start.parents):
        if (p / 'config.yaml').exists():
            return p
    return start.parent

repo_root = _find_repo_root(Path.cwd())
config_path = repo_root / 'config.yaml'
if not config_path.exists():
    raise FileNotFoundError(f'config.yaml not found at {config_path}')

with open(config_path, 'r') as fh:
    cfg = yaml.safe_load(fh) or {}

# Select appliance from list if present
selected = None
apps = cfg.get('appliances') or []
if isinstance(apps, list) and apps:
    if APPLIANCE_NAME:
        selected = next((a for a in apps if a.get('name') == APPLIANCE_NAME), None)
        if selected is None:
            raise ValueError(f"No appliance named '{APPLIANCE_NAME}' in config.yaml")
    else:
        try:
            selected = apps[int(APPLIANCE_INDEX)]
        except Exception:
            selected = apps[0]

# Resolve target and base URL
target = ((selected or {}).get('target') or cfg.get('target') or '').strip()
if not target:
    raise ValueError('config.yaml missing "target"')
BASE_URL = target if ('://' in target) else f'https://{target}'

# Resolve token or token file
token = (((selected or {}).get('token') or cfg.get('token') or '').strip())
token_file = (selected or {}).get('token_file') or cfg.get('token_file') or cfg.get('f_token')
if not token and token_file:
    tf_path = Path(token_file)
    if not tf_path.is_absolute():
        tf_path = repo_root / tf_path
    with open(tf_path, 'r') as tf:
        token = tf.read().strip()
if not token:
    raise ValueError('API token not found in config.yaml (token or token_file)')

API_VERSION = str((selected or {}).get('api_version') or cfg.get('api_version') or 'v1.14')
VERIFY_SSL = bool((selected or {}).get('verify_ssl', cfg.get('verify_ssl', True)))

# Prepare output directory consistent with CLI naming
sanitized = target.replace('.', '_').replace(':', '_').replace('/', '_')
output_dir = repo_root / f'output_{sanitized}'
output_dir.mkdir(parents=True, exist_ok=True)

print('Appliance     :', (selected or {}).get('name', '(single)'))
print('Base URL      :', BASE_URL)
print('API Version   :', API_VERSION)
print('Verify SSL    :', VERIFY_SSL)
print('Output folder :', output_dir)

## Session and helpers
Build API URLs, fetch JSON, and PATCH JSON with Authorization header.

In [ ]:
session = requests.Session()
auth_value = token if token.lower().startswith('bearer ') else f'Bearer {token}'
session.headers.update({
    'Authorization': auth_value,
    'Accept': 'application/json',
    'Content-Type': 'application/json'
})
session.verify = VERIFY_SSL

def api_url(path: str) -> str:
    base = BASE_URL.rstrip('/') + f'/api/{API_VERSION}/'
    return urljoin(base, path.lstrip('/'))

def get_json(url: str, **kwargs):
    resp = session.get(url, **kwargs)
    if resp.status_code != 200:
        print(f'Error {resp.status_code} GET {url}: {resp.text[:200]}')
        return {}
    try:
        return resp.json()
    except Exception as e:
        print(f'Failed to decode JSON: {e}')
        return {}

def patch_json(url: str, payload: dict):
    resp = session.patch(url, data=json.dumps(payload))
    ok = 200 <= resp.status_code < 300
    if not ok:
        print(f'Error {resp.status_code} PATCH {url}: {resp.text[:300]}')
        return ok, None
    try:
        return ok, (resp.json() if resp.text else None)
    except Exception:
        return ok, None

## Find 8-hour daily schedules and compute new times
We detect the cadence from `schedule.start_times`: three times equally spaced by 8 hours. The update keeps the first hour and switches to 12-hour cadence (two times per day).

In [ ]:
from typing import List, Optional

def normalize_times(times) -> List[int]:
    try:
        ints = sorted({int(t) % 24 for t in times if t is not None})
        return [t for t in ints if 0 <= t <= 23]
    except Exception:
        return []

def is_every_8_hours(times: List[int]) -> bool:
    t = normalize_times(times)
    if len(t) != 3:
        return False
    # Check circular diffs sum to 24 with each gap == 8
    diffs = [(t[(i+1)%3] - t[i]) % 24 for i in range(3)]
    return all(d == 8 for d in diffs)

def to_every_12_hours(times: List[int]) -> Optional[List[int]]:
    t = normalize_times(times)
    if not t:
        return None
    start = t[0]  # preserve earliest hour as the 'starting at'
    return [start, (start + 12) % 24]

# Fetch discovery runs
raw = get_json(api_url('discovery/runs/scheduled'))
records = (raw.get('results') if isinstance(raw, dict) else raw) or []

proposals = []
for r in records:
    schedule = (r.get('schedule') or {}) if isinstance(r, dict) else {}
    times = schedule.get('start_times') or []
    if not isinstance(times, (list, tuple)):
        continue
    if is_every_8_hours(times):
        rid = r.get('range_id') or r.get('id') or r.get('run_id')
        label = r.get('label') or r.get('name') or str(rid)
        new_times = to_every_12_hours(times)
        proposals.append({
            'run_id': rid,
            'label': label,
            'old_times': normalize_times(times),
            'new_times': new_times,
        })

df_changes = pd.DataFrame(proposals)
print(f'Total discovery runs: {len(records)}')
print(f'Candidates with 8-hour cadence: {len(df_changes)}')
df_changes.head(20)

## Apply changes (toggle above)
When `APPLY_CHANGES` is True, each candidate run is patched with `{'schedule': {'start_times': [H, H+12]}}`.
A CSV backup of proposed changes is saved under the standard output directory.

In [ ]:
updated, failed = 0, []
if not df_changes.empty:
    # Save preview/backup CSV
    out_csv = output_dir / 'schedule_updates_8_to_12.csv'
    df_out = df_changes.copy()
    df_out.insert(0, 'Discovery Instance', target)
    df_out.to_csv(out_csv, index=False)
    print(f'Preview saved to {out_csv}')

if APPLY_CHANGES and not df_changes.empty:
    for _, row in df_changes.iterrows():
        rid = row['run_id']
        new_times = row['new_times']
        if not rid or not new_times:
            continue
        url = api_url(f'/discovery/runs/scheduled/{rid}')
        payload = {'schedule': {'start_times': new_times}}
        ok, _ = patch_json(url, payload)
        if ok:
            updated += 1
        else:
            failed.append(rid)
    print(f'Updated: {updated} succeeded, {len(failed)} failed')
    if failed:
        print('Failed run IDs:', failed)
else:
    if df_changes.empty:
        print('No 8-hour schedules found; nothing to update.')
    else:
        print('APPLY_CHANGES is False; no updates performed.')